In [1]:
import os 

In [2]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
   


In [6]:
from Air_Quality_Health_Alert_System.constants import  *
from Air_Quality_Health_Alert_System.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Method to get the data ingestion configuration"""
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
from Air_Quality_Health_Alert_System import logger
from Air_Quality_Health_Alert_System.utils.common import get_size

#### Downloaded the dataset from the link mentioned in the config file 

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
    
        local_path = Path(self.config.local_data_file)

        # Ensure the parent directory exists
        local_path.parent.mkdir(parents=True, exist_ok=True)

        if not local_path.exists():
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=str(local_path)
                )
                logger.info(f"{filename} downloaded with headers: \n{headers}")
                return filename
            except Exception as e:
                logger.error(f"Failed to download file: {e}")
                raise
        else:
            logger.info(f"File already exists: {local_path}")
            return str(local_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()


except Exception as e:
    raise e

[2025-08-23 13:26:47,415: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-23 13:26:47,421: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-23 13:26:47,426: INFO: common: created directory at: artifacts]
[2025-08-23 13:26:47,432: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-23 13:26:51,242: INFO: 3398899755: artifacts\data_ingestion\data\Air_Quality_Data.csv downloaded with headers: 
Connection: close
Content-Length: 14821522
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "8a26ac482aa139ba134787511ede5c279e1d00153a28e679d6d944741cc967df"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2095:FD451:44DA:214BB:68A98957
Accept-Ranges: bytes
Date: Sat, 23 Aug 2025 09:26:48 GMT
Via: 1.1 varnish
X-Served-By: cache-dxb14